In [ ]:
import sys, os
import pandas as pd
from revoscalepy import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

RxOptions.set_option('spark.executorMem', '1g')

In [ ]:
dirpath = RxOptions.get_option("sampleDataDir")

# CSV File (Raw Data for one month)
airlineDemoPathCSV = os.path.join(dirpath, "AirlineDemoSmall.csv")

df = pd.read_csv(airlineDemoPathCSV, dtype={'CRSDepTime': float,'DayOfWeek': 'category'})
df = df[~df['ArrDelay'].apply(lambda x: x == 'M')]
df = df.astype({"ArrDelay": int})


In [ ]:
df.dtypes

In [ ]:
cc = rx_spark_connect(master='spark://centos-7-spark.shared:7077', 
                      executor_cores = 1, console_output = True, interop='pyspark', reset=True)

In [ ]:
sc = rx_get_pyspark_connection(cc)
spark = SparkSession(sc)
spark_df = spark.createDataFrame(df)
trainDS = RxSparkDataFrame(data = spark_df)

In [ ]:
linmod = rx_lin_mod("ArrDelay ~ CRSDepTime", data=trainDS)

In [ ]:
linmod

In [ ]:
rx_spark_disconnect(cc)